In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(1, '/home/jw3514/Work/CellType_Psy/src')
from CellType_PSY import *
import scanpy as sc
HGNC, ENSID2Entrez, GeneSymbol2Entrez, Entrez2Symbol = LoadGeneINFO()

import requests
import SimpleITK as sitk
import pathlib

from sklearn.cluster import DBSCAN
from scipy.spatial import ConvexHull
import alphashape
from alpha_shapes.boundary import Boundary, get_boundaries

from descartes import PolygonPatch
from alpha_shapes import Alpha_Shaper, plot_alpha_shape

In [ ]:
def get_region_shape(section, reg, level="structure"):
    if level == "structure":
        sub_section = section[section["parcellation_structure"]==reg]
    elif level == "division":
        sub_section = section[section["parcellation_division"]==reg]
    points = sub_section[["x_reconstructed", "y_reconstructed"]].values
    print(points.shape)
    shaper = Alpha_Shaper(points)
    alpha_opt, alpha_shape = shaper.optimize()
    alpha_shape = shaper.get_shape(alpha=alpha_opt * 0.6)
    
    condition = points[:, 0] < 6
    subset_data = points[condition]
    center = subset_data.mean(axis=0)
    return alpha_shape, center
def add_region_bound(plt, alpha_shape, center, reg):
    for bound in get_boundaries(alpha_shape):
        data = bound._exterior
        plt.plot(data[:,0], data[:,1], color='black', linestyle='dashed', linewidth=1.5)
        holes = bound._holes
        for hole in holes:
            plt.plot(hole[:,0], hole[:,1], color='black', linestyle='dashed', linewidth=1.5)
        plt.text(x=center[0] - 0.5, y=center[1], s=reg)

In [ ]:
ClusterAnn = pd.read_excel("../../data/Allen_Mouse_Brain_Cell_Atlas/SuppTables/41586_2023_6812_MOESM8_ESM.xlsx",
                          sheet_name = "cluster_annotation", index_col="cluster_id_label")

In [ ]:
MERFISH = pd.read_csv("/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/MERFISH/MERFISH.ISH_Annot.clean.csv", index_col=0)

In [ ]:
MERFISH[MERFISH["parcellation_structure"]=="CA1"]["brain_section_label"].value_counts()

In [ ]:
CA1_MERFISH = MERFISH[MERFISH["parcellation_structure"]=="CA1"]

In [ ]:
CA1_MERFISH.shape

In [ ]:
brain_section = 'C57BL6J-638850.30'
pred = (CA1_MERFISH['brain_section_label'] == brain_section)
section = CA1_MERFISH[pred]
print(len(section))

In [ ]:
section.columns.values

In [ ]:
section.head(2)

In [ ]:
section["subclass"].value_counts()/section["subclass"].shape[0]

In [ ]:
def plot_section( xx=None, yy=None, cc=None, val=None, pcmap=None, 
                 overlay=None, extent=None, bcmap=plt.cm.Greys_r, alpha=1.0,
                 fig_width = 6, fig_height = 6 ) :
    
    fig, ax = plt.subplots(dpi=480)
    fig.set_size_inches(fig_width, fig_height)

    if xx is not None and yy is not None and pcmap is not None :
        plt.scatter(xx,yy,s=0.5,c=val,marker='.',cmap=pcmap)
    elif xx is not None and yy is not None and cc is not None :
        plt.scatter(xx,yy,s=0.5,color=cc,marker='.',zorder=1)   
        
    if overlay is not None and extent is not None and bcmap is not None :
        plt.imshow(overlay, cmap=bcmap, extent=extent,alpha=alpha,zorder=2)
        
    ax.set_ylim(11,0)
    ax.set_xlim(0,11)
    ax.axis('equal')
    ax.set_xticks([])
    ax.set_yticks([])
    
    return fig, ax

In [ ]:
sns.set(style="whitegrid", context="talk")
plt.figure(dpi=480, figsize=(10, 8))
CTs = ["016 CA1-ProS Glut", "023 SUB-ProS Glut", 
                  "053 Sst Gaba", "052 Pvalb Gaba", 
                  "046 Vip Gaba", "049 Lamp5 Gaba"]
Colors = ["#542788", "#998ec3", "#b2df8a", "#33a02c",
         "#e31a1c", "#ff7f00"]
color_dict = dict(zip(CTs, Colors))
for _subclass in CTs:
    _setction = section[section["subclass"]==_subclass]
    plt.scatter(_setction["x_reconstructed"], _setction["y_reconstructed"], color=color_dict[_subclass], 
                label=_subclass, s=30, alpha=0.5)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
(section["subclass"].value_counts()/section["subclass"].shape[0])[CTs]

In [ ]:
for n in np.arange(28, 42):
    try:
        print(n)
        brain_section = 'C57BL6J-638850.{}'.format(n)
        pred = (CA1_MERFISH['brain_section_label'] == brain_section)
        section = CA1_MERFISH[pred]
        print((section["subclass"].value_counts()/section["subclass"].shape[0])[CTs])
    except:
        continue